# Keras tutorial: efficient network construction using model subclassing API
### Here, we construct a network composed of repeated 'blocks' of operatons.
### We define each block with a class, and use it to construct the network in a code efficient manner. 


### Import dependencies

In [1]:
import numpy as np
import keras
import tensorflow as tf
import tensorflow.keras as tk
from tensorflow.keras import layers

print(tf. __version__) 
print(keras.__version__)

2.9.0
2.9.0


### Load in data

In [2]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# Load the data and split it between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)



# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train_1 = x_train[1:1000,:,:]


x_val_1 = x_train[1000:1150,:,:]

x_test_1 = x_train[1150:1250,:,:]

y_train_1 = y_train[1:1000]

y_val_1 = y_train[1000:1150]

y_test_1 = y_train[1150:1250]
print("x_train shape:", x_train_1.shape)
print(x_train_1.shape[0], "train samples")
print(x_test_1.shape[0], "test samples")

x_train shape: (999, 28, 28, 1)
999 train samples
100 test samples


### Set hyperparameters

In [3]:
EPOCHS=30
BATCH_SIZE = 5

In [4]:
class ConvBlock(layers.Layer):
    # class instances are manipulated by the functions in the class
    def __init__(self, filters=32, kernel_size = 3):
        super(ConvBlock,self).__init__()
        self.conv = layers.Conv2D(filters, kernel_size, padding='same')
        self.bn = layers.BatchNormalization()
        
    # define the operations peformed by the block
    def call(self, input_tensor, training=False):
        x = self.conv(input_tensor)
        x = self.bn(x,training=training) # Batchnorm uses training as an input.
        x = layers.Activation(keras.activations.relu)(x)
        return x
    
class ConvChainModule(layers.Layer):
    # class instances are manipulated by the functions in the class
    def __init__(self, filters=32, kernel_size = 3):
        super(ConvChainModule,self).__init__()
        self.conv_chain1 = ConvBlock()
        self.conv_chain2 = ConvBlock()
        self.conv_chain3 = ConvBlock()
        
        
    # define the operations peformed by the block
    def call(self, input_tensor, training=False):
        #x = layers.Conv2D(10, 3, padding='same')(input_tensor)
        x = self.conv_chain1(input_tensor)
        x = self.conv_chain2(x)
        x = self.conv_chain3(x)
        return x
        

In [5]:
net_input_1 = keras.Input(shape=(28, 28, 1), name="img")
x = ConvChainModule()(net_input_1)
x = layers.MaxPooling2D(pool_size=(2, 2))(x)
x = ConvChainModule()(x)
x = layers.MaxPooling2D(pool_size=(2, 2))(x)
x = ConvChainModule()(x)
x = layers.MaxPooling2D(pool_size=(2, 2))(x)
x = layers.Conv2D(filters=1, kernel_size = 3, padding='same')(x)
x = layers.Flatten()(x)
net_output_= layers.Dense(num_classes, activation="softmax")(x)


In [6]:
class MyModel(object):
    # initialise instance attributes in __init_ - these are manipulated by the functions in the class.
    # Keras model takes variable input_shape as an input, so we include this as an input. 
    def __init__(self,input_shape):
        # super below runs the __init__ of parent class (in this case, object).
        # we don't really need this, but it's kept to illustrate potential functionality.
        super(MyModel,self).__init__()
        self.input_shape = input_shape
        self.net_1 = keras.Model(net_input_1, net_output_, name="net_1")

    def fit(self, x,y,x_val,y_val):
        class CustomSaver(keras.callbacks.Callback):
            def on_epoch_end(self, epoch, logs={}):
                # we use self.model because we are inhereting the keras.callbacks.Callback class which uses the model attribute
                self.model.save("combined_net_{}.hd5".format(epoch))  
        self.net_1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        from keras.callbacks import CSVLogger
        csv_logger = CSVLogger('pretrain_log_combined.csv')
        saver = CustomSaver()
        self.net_1.fit(x, y, batch_size=BATCH_SIZE, epochs=EPOCHS, callbacks=[csv_logger,saver],validation_data=(x_val, y_val))
        
        


In [7]:
our_model = MyModel(input_shape = (28,28,1))
our_model.net_1.summary()
our_model.fit(x_train_1,y_train_1,x_val_1,y_val_1)

Model: "net_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 img (InputLayer)            [(None, 28, 28, 1)]       0         
                                                                 
 conv_chain_module (ConvChai  (None, 28, 28, 32)       19200     
 nModule)                                                        
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 32)       0         
 )                                                               
                                                                 
 conv_chain_module_1 (ConvCh  (None, 14, 14, 32)       28128     
 ainModule)                                                      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 32)         0         
 2D)                                                         

INFO:tensorflow:Assets written to: combined_net_0.hd5\assets


INFO:tensorflow:Assets written to: combined_net_0.hd5\assets


200/200 [==============================] - 10s 42ms/step - loss: 1.0565 - accuracy: 0.6877 - val_loss: 3.9235 - val_accuracy: 0.1467
Epoch 2/30
200/200 [==============================] - ETA: 0s - loss: 0.3186 - accuracy: 0.9129

INFO:tensorflow:Assets written to: combined_net_1.hd5\assets


...

INFO:tensorflow:Assets written to: combined_net_28.hd5\assets


200/200 [==============================] - 8s 42ms/step - loss: 0.1038 - accuracy: 0.9710 - val_loss: 0.1304 - val_accuracy: 0.9333
Epoch 30/30
200/200 [==============================] - ETA: 0s - loss: 0.0191 - accuracy: 0.9930

INFO:tensorflow:Assets written to: combined_net_29.hd5\assets


INFO:tensorflow:Assets written to: combined_net_29.hd5\assets


200/200 [==============================] - 8s 40ms/step - loss: 0.0191 - accuracy: 0.9930 - val_loss: 0.3419 - val_accuracy: 0.8933


In [8]:
score = our_model.net_1.evaluate(x_test_1, y_test_1, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.203872948884964
Test accuracy: 0.9399999976158142
